# Data Scraping
Author: Keanna Knebel

Date: 2020/07/11

In [171]:
# Import packages
import requests
import pandas as pd
import re
import time

from bs4 import BeautifulSoup

In [169]:
# initialize empty dataframe
df = pd.DataFrame(columns =['Year', 'Week', 'Date', 'Favorite', 'Spread', 'Underdog'])

# scrap website for each of the 17 NFL weeks
for week in range(1,18):
    if week == 17:
        page = requests.get(f"http://www.footballlocks.com/nfl_odds_week_{week}.shtml#Closing%20NFL%20Odds%20Week%20{week},%202016-2017")
    else:
        page = requests.get(f"http://www.footballlocks.com/nfl_odds_week_{week}.shtml#Closing%20NFL%20Odds%20Week%20{week},%202016")
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # select weekly data tables from web page
    weekly_tables = soup.find_all('table', attrs={'width':644})
    
    # initialize columns for dataframe
    date = []
    favorite = []
    spread = []
    underdog = []
    df_year = []
    
    for table in weekly_tables:
        
        try:
            year = table.a['name'][-4:]
        except:
            year = year
        
        for rows in table.find_all('tr'):
            cols = rows.find_all('td')

            if len(cols) == 6:
                df_year.append(year)
                date.append(cols[0].get_text())
                favorite.append(cols[1].get_text())
                spread.append(cols[2].get_text())
                underdog.append(cols[3].get_text())

    # create dataframe from the lists of columns
    new_df = pd.DataFrame(list(zip(df_year, date, favorite, spread, underdog)), 
                       columns =['Year', 'Date', 'Favorite', 'Spread', 'Underdog'])
    new_df['Week'] = week
    
    # clean up dataframe by removing blank and title rows
    new_df = new_df.drop_duplicates(subset=['Date', 'Favorite', 'Spread', 'Underdog'], keep=False, ignore_index=True)
    
    # combine dataframes into one
    df = df.append(new_df)
    
    time.sleep(5)

In [170]:
df

,Year,Week,Date,Favorite,Spread,Underdog
0,2019,1,9/5 8:20 ET,At Chicago,-3,Green Bay
1,2019,1,9/8 1:03 ET,At Minnesota,-3.5,Atlanta
2,2019,1,9/8 1:03 ET,At Philadelphia,-10,Washington
3,2019,1,9/8 1:03 ET,At NY Jets,-2.5,Buffalo
4,2019,1,9/8 1:03 ET,Baltimore,-7,At Miami
...,...,...,...,...,...,...
201,2006,17,12/31 4:15 ET,At Philadelphia,-6.5,Atlanta
202,2006,17,12/31 1:00 ET,At Tampa Bay,-3,Seattle
203,2006,17,12/31 1:00 ET,At Tennessee,-3.5,New England
204,2006,17,12/31 4:15 ET,At Denver,-11,San Francisco


In [ ]:
## TO DO ##

# clean date column
# set 'pk' to 0 values for spread
# remove 'at' from team names